## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Modelado

---

En esta libreta se comienzan a ejecutar las actividades correspondientes a la fase de modelado. 

---

### Carga de librerias a utilizar

In [27]:
# Se importan las librerias a utilizar
import pandas as pd
import numpy as np

# Para registro de toda la experimentación
import mlflow

# Para la generación de los sets de train - test
from sklearn.model_selection import train_test_split

#Importaciones varias para la evaluación
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Lectura del dataset generado previamente

In [28]:
# Se hace la lectura del dataset del seminario y se obtiene su cabecera
datos = pd.read_csv('../../data/final/datos_finales.csv', sep=';')
datos.head()

,edad,antiguedad_empleado,antiguedad_cliente,ingresos,pct_ingreso,tasa_interes,gastos_ult_12m,operaciones_ult_12m,estado_civil_N,estado_credito_N,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,falta_pago,estado_cliente,genero,limite_credito_tc,nivel_educativo,personas_a_cargo
0,menor 25,NaN,2y_a_4y,50k_a_100k,40_a_60,15p_a_20p,1k_a_4k,15k_a_30k,C,P,35000,3,ALQUILER,False,Y,ACTIVO,M,4010.0,UNIVERSITARIO_COMPLETO,2.0
1,menor 25,5_a_10,2y_a_4y,hasta_20k,hata_20,7p_a_15p,1k_a_4k,30k_a_50k,C,C,1000,2,PROPIA,False,N,ACTIVO,M,12691.0,SECUNDARIO_COMPLETO,3.0
2,menor 25,5_a_10,2y_a_4y,50k_a_100k,40_a_60,15p_a_20p,1k_a_4k,15k_a_30k,C,P,35000,2,ALQUILER,False,N,ACTIVO,M,3418.0,UNIVERSITARIO_COMPLETO,3.0
3,menor 25,5_a_10,mayor_4y,50k_a_100k,40_a_60,7p_a_15p,1k_a_4k,15k_a_30k,C,P,35000,4,ALQUILER,False,Y,ACTIVO,M,9095.0,DESCONOCIDO,1.0
4,menor 25,menor 10,2y_a_4y,hasta_20k,20_a_40,7p_a_15p,1k_a_4k,15k_a_30k,N,P,2500,2,PROPIA,False,N,ACTIVO,F,3313.0,SECUNDARIO_COMPLETO,4.0


### Revisión de meta-datos

In [29]:
def reporte_descripcion_dataset(df):
    columnas = df.columns
    print("Columnas del dataset:\n")
    for col in columnas:
        print(col)
    print(f"\nCantidad de filas: {df.shape[0]}")

print("Descripción del dataset 'datos_completos'")
reporte_descripcion_dataset(datos)

Descripción del dataset 'datos_completos'
Columnas del dataset:

edad
antiguedad_empleado
antiguedad_cliente
ingresos
pct_ingreso
tasa_interes
gastos_ult_12m
operaciones_ult_12m
estado_civil_N
estado_credito_N
importe_solicitado
duracion_credito
situacion_vivienda
objetivo_credito
falta_pago
estado_cliente
genero
limite_credito_tc
nivel_educativo
personas_a_cargo

Cantidad de filas: 10108


### Armado de los sets de entrenamiento y testeo

In [30]:
# Se separan los atributos de datos de los de la etiqueta del dataset

labels = datos.falta_pago.values

features = datos[['operaciones_ult_12m', 'gastos_ult_12m', 'limite_credito_tc',
       'antiguedad_cliente', 'tasa_interes', 'ingresos', 'pct_ingreso',
       'antiguedad_empleado', 'edad', 'importe_solicitado', 'duracion_credito',
       'situacion_vivienda', 'objetivo_credito', 'estado_credito_N',
       'estado_civil_N', 'estado_cliente', 'genero',
       'nivel_educativo', 'personas_a_cargo']]

print(f"Vista de los datos del atributo target: \n{labels[:10]}")
print("-"*50)
print("Vista parcial de los datos sin el atributo target:")
features.head()

Vista de los datos del atributo target: 
['Y' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N']
--------------------------------------------------
Vista parcial de los datos sin el atributo target:


,operaciones_ult_12m,gastos_ult_12m,limite_credito_tc,antiguedad_cliente,tasa_interes,ingresos,pct_ingreso,antiguedad_empleado,edad,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,estado_credito_N,estado_civil_N,estado_cliente,genero,nivel_educativo,personas_a_cargo
0,15k_a_30k,1k_a_4k,4010.0,2y_a_4y,15p_a_20p,50k_a_100k,40_a_60,NaN,menor 25,35000,3,ALQUILER,False,P,C,ACTIVO,M,UNIVERSITARIO_COMPLETO,2.0
1,30k_a_50k,1k_a_4k,12691.0,2y_a_4y,7p_a_15p,hasta_20k,hata_20,5_a_10,menor 25,1000,2,PROPIA,False,C,C,ACTIVO,M,SECUNDARIO_COMPLETO,3.0
2,15k_a_30k,1k_a_4k,3418.0,2y_a_4y,15p_a_20p,50k_a_100k,40_a_60,5_a_10,menor 25,35000,2,ALQUILER,False,P,C,ACTIVO,M,UNIVERSITARIO_COMPLETO,3.0
3,15k_a_30k,1k_a_4k,9095.0,mayor_4y,7p_a_15p,50k_a_100k,40_a_60,5_a_10,menor 25,35000,4,ALQUILER,False,P,C,ACTIVO,M,DESCONOCIDO,1.0
4,15k_a_30k,1k_a_4k,3313.0,2y_a_4y,7p_a_15p,hasta_20k,20_a_40,menor 10,menor 25,2500,2,PROPIA,False,P,N,ACTIVO,F,SECUNDARIO_COMPLETO,4.0


In [31]:
# Se pasa a binarizar los atributos

data = pd.get_dummies(features)
data.head()

,limite_credito_tc,importe_solicitado,duracion_credito,objetivo_credito,personas_a_cargo,operaciones_ult_12m_15k_a_30k,operaciones_ult_12m_30k_a_50k,operaciones_ult_12m_50k_a_75k,operaciones_ult_12m_75k_a_100k,operaciones_ult_12m_mayor_100k,operaciones_ult_12m_menor_15k,gastos_ult_12m_1k_a_4k,gastos_ult_12m_4k_a_6k,gastos_ult_12m_6k_a_8k,gastos_ult_12m_8k_a_10k,gastos_ult_12m_menor_1k,antiguedad_cliente_2y_a_4y,antiguedad_cliente_mayor_4y,antiguedad_cliente_menor_2y,tasa_interes_15p_a_20p,tasa_interes_7p_a_15p,tasa_interes_hasta_7p,tasa_interes_mayor_20p,ingresos_20k_a_50k,ingresos_50k_a_100k,ingresos_hasta_20k,ingresos_mayor_100k,pct_ingreso_20_a_40,pct_ingreso_40_a_60,pct_ingreso_hata_20,pct_ingreso_mayor_60,antiguedad_empleado_5_a_10,antiguedad_empleado_mayor_10,antiguedad_empleado_menor 10,edad_25_a_30,edad_menor 25,situacion_vivienda_ALQUILER,situacion_vivienda_HIPOTECA,situacion_vivienda_OTROS,situacion_vivienda_PROPIA,estado_credito_N_C,estado_credito_N_P,estado_civil_N_C,estado_civil_N_D,estado_civil_N_N,estado_civil_N_S,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,4010.0,35000,3,False,2.0,True,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,True,True,False,False,False,True,False,False,True,False,False,False,False,True,False
1,12691.0,1000,2,False,3.0,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,True,False,True,False,False,False,True,False,False,False,True,True,False,True,False,False,False,True,False,False,True,False,False,False,True,False,False
2,3418.0,35000,2,False,3.0,True,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,True,False,False,False,False,True,True,False,False,False,True,False,False,True,False,False,False,False,True,False
3,9095.0,35000,4,False,1.0,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,True,False,False,False,False,True,True,False,False,False,True,False,False,True,True,False,False,False,False,False
4,3313.0,2500,2,False,4.0,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,True,False,False,False,True,False,True,False,False,True,False,True,False,True,False,False,False,False,True,False,False


In [32]:
# Para evitar problemas con mlfow se hace una conversión de integer a double 
# de los atributos: 'importe_solicitado', 'duracion_credito', 'estado_credito'

# Se filtran las columnas del tipo integer
integer_columns = data.select_dtypes(include='int').columns

# Convert integer columns to double
data[integer_columns] = data[integer_columns].astype('float')

In [33]:
# Se reparten los datos disponibles en conjuntos para entrenamiento y testeo

train_data, test_data, train_labels, test_labels = train_test_split(data, labels) 

# Toma por defecto 75% (entrenamiento) - 25% (testeo)

# Vista de los datos de entrenamiento para una fila

print(f"Un registro de entrenamiento: \n{train_data.iloc[1]}")
print("-"*50)
print(f"Label del registro: \n{train_labels[1]}\n")

print("#"*50)
print("#"*50)

# Vista de los datos de testeo para una fila

print(f"\nUn registro de testeo: \n{test_data.iloc[1]}")
print("-"*50)
print(f"Label del registro: \n{test_labels[1]}")

Un registro de entrenamiento: 
limite_credito_tc                            5535.0
importe_solicitado                          13300.0
duracion_credito                                4.0
objetivo_credito                              False
personas_a_cargo                                4.0
operaciones_ult_12m_15k_a_30k                 False
operaciones_ult_12m_30k_a_50k                  True
operaciones_ult_12m_50k_a_75k                 False
operaciones_ult_12m_75k_a_100k                False
operaciones_ult_12m_mayor_100k                False
operaciones_ult_12m_menor_15k                 False
gastos_ult_12m_1k_a_4k                         True
gastos_ult_12m_4k_a_6k                        False
gastos_ult_12m_6k_a_8k                        False
gastos_ult_12m_8k_a_10k                       False
gastos_ult_12m_menor_1k                       False
antiguedad_cliente_2y_a_4y                     True
antiguedad_cliente_mayor_4y                   False
antiguedad_cliente_menor_2y      

In [34]:
# Configuración de la evaluación

def evaluar_modelo(modelo, test_data, test_labels):
    prediction = modelo.predict(test_data)
    print('Rendimiento obtenido:',accuracy_score(test_labels,prediction))
    print('Reporte de indicadores:\n',classification_report(test_labels,prediction))
    print('Matriz de confusión:')
    cm = confusion_matrix(test_labels, prediction, labels=modelo.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                   display_labels=modelo.classes_)
    disp.plot()
    plt.show()

In [35]:
# Para obtener todos los parámetros y no solo los modificados

from sklearn import set_config

set_config(print_changed_only=False)

### Generación de modelos

#### ZeroR

In [36]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier()
clf.fit(train_data, train_labels)

TypeError: BaseEstimator._repr_mimebundle_() takes 1 positional argument but 3 were given

In [37]:
prediction = clf.predict(test_data)

print('Rendimiento obtenido:',accuracy_score(test_labels, prediction))
print('Vista de una muestra de valores de predicción y datos de testeo:')
print(prediction[:10])
print(test_labels[:10])

Rendimiento obtenido: 0.814404432132964
Vista de una muestra de valores de predicción y datos de testeo:
['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N']
['N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N']


----
Se pasa a configurar la serie de experimentos en **mlflow**

In [38]:
# Se establece un nombre para identificar la serie de experimentos

mlflow.set_experiment(experiment_name="Prueba #1")
mlflow.autolog()

2023/12/09 16:00:24 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


#### Regresión Logística

In [39]:
from sklearn.linear_model import LogisticRegression

with mlflow.start_run(run_name='Logistic Regression'):
    lreg = LogisticRegression(solver = 'liblinear')

    lreg.fit(train_data, train_labels)

2023/12/09 16:00:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [ ]:
evaluar_modelo(lreg, test_data, test_labels)

#### KNN

In [40]:
from sklearn.neighbors import KNeighborsClassifier

with mlflow.start_run(run_name='KNN'):
    knn = KNeighborsClassifier(n_neighbors=50, 
                               algorithm = 'ball_tree', 
                               leaf_size = 25)

    knn.fit(train_data, train_labels)

2023/12/09 16:00:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [ ]:
evaluar_modelo(knn, test_data, test_labels)

#### Árboles de Decisión (TDIDT)

In [41]:
from sklearn.tree import DecisionTreeClassifier

with mlflow.start_run(run_name='Decision Tree'):
    dtc = DecisionTreeClassifier(max_depth=3, 
                                 criterion='entropy', 
                                 min_samples_split=10)

    dtc.fit(train_data, train_labels)

2023/12/09 16:00:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [ ]:
evaluar_modelo(dtc, test_data, test_labels)

#### Métodos de ensamblado de modelos: RandomForest

In [42]:
from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run(run_name='Random Forest'):
    rndf = RandomForestClassifier(n_estimators=10)
    rndf.fit(train_data, train_labels)

2023/12/09 16:00:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [ ]:
evaluar_modelo(rndf, test_data, test_labels)

Prueba #2

In [43]:
mlflow.set_experiment(experiment_name="Prueba #2")


<Experiment: artifact_location='file:///home/sebastian/Documentos/programas/Maestria/Metodologias%20de%20gestion%20y%20dise%C3%B1o%20proyecto%20big%20data/Git/Metodologias-Agiles/notebooks/Modelado-Evaluacion/mlruns/122100842435769979', creation_time=1702149433091, experiment_id='122100842435769979', last_update_time=1702149433091, lifecycle_stage='active', name='Prueba #2', tags={}>


 Arbol decision#2

In [44]:
with mlflow.start_run(run_name='Decision Tree (#2)'):
    dtc = DecisionTreeClassifier(max_depth=5, 
                                 criterion='entropy', 
                                 min_samples_split=15)

    dtc.fit(train_data, train_labels)

2023/12/09 16:01:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [51]:
print("Salida de resultado de la evaluación correspondientes a la Prueba #2")
evaluar_modelo(dtc, test_data, test_labels)

Salida de resultado de la evaluación correspondientes a la Prueba #2


/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
2023/12/09 16:09:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:

Rendimiento obtenido: 0.8686189157103285
Reporte de indicadores:
               precision    recall  f1-score   support

           N       0.91      0.93      0.92      2058
           Y       0.67      0.58      0.62       469

    accuracy                           0.87      2527
   macro avg       0.79      0.76      0.77      2527
weighted avg       0.86      0.87      0.86      2527

Matriz de confusión:


 Regresion logistica(#2)

In [45]:
from sklearn.linear_model import LogisticRegression

with mlflow.start_run(run_name='Logistic Regression#2'):
    lreg2 = LogisticRegression(solver = 'newton-cholesky')

    lreg2.fit(train_data, train_labels)

2023/12/09 16:01:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [52]:
evaluar_modelo(lreg2, test_data, test_labels)

/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
2023/12/09 16:10:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:

Rendimiento obtenido: 0.8674317372378314
Reporte de indicadores:
               precision    recall  f1-score   support

           N       0.91      0.93      0.92      2058
           Y       0.67      0.58      0.62       469

    accuracy                           0.87      2527
   macro avg       0.79      0.75      0.77      2527
weighted avg       0.86      0.87      0.86      2527

Matriz de confusión:


K-NN #2

In [46]:
from sklearn.neighbors import KNeighborsClassifier

with mlflow.start_run(run_name='KNN #2'):
    knn2 = KNeighborsClassifier(n_neighbors=70, 
                               algorithm = 'kd_tree', 
                               leaf_size = 35)

    knn2.fit(train_data, train_labels)

2023/12/09 16:01:44 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [53]:
evaluar_modelo(knn2, test_data, test_labels)

/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
2023/12/09 16:10:35 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:

Rendimiento obtenido: 0.814404432132964


/home/sebastian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Reporte de indicadores:
               precision    recall  f1-score   support

           N       0.81      1.00      0.90      2058
           Y       0.00      0.00      0.00       469

    accuracy                           0.81      2527
   macro avg       0.41      0.50      0.45      2527
weighted avg       0.66      0.81      0.73      2527

Matriz de confusión:


Ramdon forest #2

In [49]:
from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run(run_name='Random Forest#2'):
    rndf2 = RandomForestClassifier(n_estimators=12)
    rndf2.fit(train_data, train_labels)

2023/12/09 16:02:53 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(

In [50]:
evaluar_modelo(rndf2, test_data, test_labels)

/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
2023/12/09 16:03:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:

Rendimiento obtenido: 0.8698060941828255
Reporte de indicadores:
               precision    recall  f1-score   support

           N       0.89      0.96      0.92      2058
           Y       0.72      0.49      0.59       469

    accuracy                           0.87      2527
   macro avg       0.80      0.72      0.75      2527
weighted avg       0.86      0.87      0.86      2527

Matriz de confusión:


Prueba #3

Arbol decision#3


In [54]:
with mlflow.start_run(run_name='Decision Tree (#3)'):
    dtc = DecisionTreeClassifier(max_depth=8, 
                                 criterion='gini', 
                                 min_samples_split=18)

    dtc.fit(train_data, train_labels)

2023/12/09 16:21:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [55]:

print("Salida de resultado de la evaluación correspondientes a la Prueba #3")
evaluar_modelo(dtc, test_data, test_labels)

Salida de resultado de la evaluación correspondientes a la Prueba #3


/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
2023/12/09 16:21:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:

Rendimiento obtenido: 0.8634744756628413
Reporte de indicadores:
               precision    recall  f1-score   support

           N       0.90      0.93      0.92      2058
           Y       0.66      0.56      0.60       469

    accuracy                           0.86      2527
   macro avg       0.78      0.74      0.76      2527
weighted avg       0.86      0.86      0.86      2527

Matriz de confusión:


 Regresion logistica(#3)

In [56]:
from sklearn.linear_model import LogisticRegression

with mlflow.start_run(run_name='Logistic Regression#3'):
    lreg3 = LogisticRegression(solver = 'newton-cg')

    lreg3.fit(train_data, train_labels)

2023/12/09 16:23:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [57]:
evaluar_modelo(lreg3, test_data, test_labels)

/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
2023/12/09 16:23:21 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:

Rendimiento obtenido: 0.8674317372378314
Reporte de indicadores:
               precision    recall  f1-score   support

           N       0.91      0.93      0.92      2058
           Y       0.67      0.57      0.62       469

    accuracy                           0.87      2527
   macro avg       0.79      0.75      0.77      2527
weighted avg       0.86      0.87      0.86      2527

Matriz de confusión:


K-NN#3

In [61]:
from sklearn.neighbors import KNeighborsClassifier

with mlflow.start_run(run_name='KNN #3'):
    knn3 = KNeighborsClassifier(n_neighbors=75, 
                               algorithm = 'brute', 
                               leaf_size = 40)

    knn3.fit(train_data, train_labels)

2023/12/09 16:25:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pr

In [62]:
evaluar_modelo(knn3, test_data, test_labels)

/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
2023/12/09 16:25:43 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:

Rendimiento obtenido: 0.814404432132964


/home/sebastian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Reporte de indicadores:
               precision    recall  f1-score   support

           N       0.81      1.00      0.90      2058
           Y       0.00      0.00      0.00       469

    accuracy                           0.81      2527
   macro avg       0.41      0.50      0.45      2527
weighted avg       0.66      0.81      0.73      2527

Matriz de confusión:


Ramdon Forest #3

In [63]:
from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run(run_name='Random Forest#3'):
    rndf3 = RandomForestClassifier(n_estimators=14)
    rndf3.fit(train_data, train_labels)

2023/12/09 16:27:24 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [64]:
evaluar_modelo(rndf3, test_data, test_labels)

/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sebastian/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
2023/12/09 16:27:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sebastian/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:

Rendimiento obtenido: 0.8729719034428176
Reporte de indicadores:
               precision    recall  f1-score   support

           N       0.89      0.96      0.92      2058
           Y       0.73      0.51      0.60       469

    accuracy                           0.87      2527
   macro avg       0.81      0.73      0.76      2527
weighted avg       0.86      0.87      0.86      2527

Matriz de confusión:


#

#### TODO: posibilidades de mejora de la libreta

* Evaluación del clasificador a través de validación cruzada
* Ejecutar pruebas que contemplen datos con mayor o menor grado de transformaciones aplicadas 
* Implementar clasificación con otras técnicas (por ejemplo: redes neuronales, SVM, entre otras)
* Evaluar la combinación de técnicas, por ejemplo: clustering y sobre esos resultados aplicación de árboles de decisión para detectar patrones de agrupamiento y ahí relacionarlo con la situación de los clientes.
* Integrar técnicas de extracción de reglas de asociación (con mayor "legibilidad" para usuarios no-técnicos)
* Entre otras opciones

----

### Prueba de despliegue (utilzando datos nuevos)

In [ ]:
datos_nuevos = pd.read_csv('../../data/final/datos_nuevos.csv', sep=';')
datos_nuevos.head(5)

**Importante**: recordar que si se implementan operaciones de transformación sobre los datos será necesario (posiblemente) replicarlas sobre los datos nuevos para que ambos datasets sean compatibles.

#### Se tienen que aplicar las adaptaciones realizadas sobre los datos de entrenamiento

In [ ]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = datos_nuevos.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil')

# Columna: estado_credito
cambios_estado_credito = {
    0: 'C',
    1 : 'P',
}

estado_credito_N = datos_nuevos.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito')

# df_final = pd.concat([estado_civil_N, estado_credito_N, df_integrado], axis=1)
# df_final.head()

# Antiguedad del empleado

etiquetas_a_e = ['menor_10', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
valor_para_nan = 'NA'
antiguedad_empleados_N = pd.cut(datos_nuevos['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

# antiguedad_empleados_N.value_counts()

# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(datos_nuevos['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

# edad_N.value_counts()

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(datos_nuevos['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

# pct_ingreso_N.value_counts()

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(datos_nuevos['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

# ingresos_N.value_counts()

# tasa_interes

etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
tasa_interes_N = pd.cut(datos_nuevos['tasa_interes'], 
                                bins=rangos_t_i, 
                                labels=etiquetas_t_i)

# tasa_interes_N.value_counts()

# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(datos_nuevos['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

# antiguedad_cliente_N.value_counts()

# limite_credito_tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(datos_nuevos['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)

# limite_credito_tc_N.value_counts()

# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '2k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(datos_nuevos['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

# gastos_ult_12m_N.value_counts()

# operaciones_ult_12m

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(datos_nuevos['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)

# operaciones_ult_12m_N.value_counts()

col_eliminar_final = [
              'edad',
              'antiguedad_empleado',
              'antiguedad_cliente', 
              'ingresos',
              'pct_ingreso', 
              'tasa_interes',
              'gastos_ult_12m', 
              'limite_credito_tc', 
              'operaciones_ult_12m',
              'nivel_tarjeta',
              'estado_civil',
              'estado_credito',
              'id_cliente']

datos_nuevos.drop(col_eliminar_final, inplace=True, axis=1)


df_nuevo = pd.concat([operaciones_ult_12m_N, gastos_ult_12m_N, limite_credito_tc_N, antiguedad_cliente_N, tasa_interes_N, ingresos_N, pct_ingreso_N, antiguedad_empleados_N, edad_N, estado_civil_N, estado_credito_N, datos_nuevos], axis=1)
df_nuevo.head(5)

In [ ]:
nuevos_codif = pd.get_dummies(df_nuevo)
nuevos_codif.info()

In [ ]:
# Dado que puede haber diferencias con el set de entrenamiento se emparejan

columnas_faltantes = set(data.columns) - set(nuevos_codif.columns)
display(columnas_faltantes)
for columna in columnas_faltantes:
    nuevos_codif[columna] = 0 # Se agregan las columnas faltantes con valor 0

print('Datos nuevos: ' + str(len(nuevos_codif.columns)))
print('Datos entrenamiento: ' + str(len(data.columns)))
print(set(data.columns) - set(nuevos_codif.columns))

In [ ]:
nuevos_codif = nuevos_codif[data.columns]
nuevos_codif.columns

In [ ]:
# Se hace la predicción con los nuevos datos

prediccion_nuevos = dtc.predict(nuevos_codif)

prediccionDF = pd.DataFrame(prediccion_nuevos)

In [ ]:
# Se agrega la nueva columna
datos_nuevos['prediccion_mora'] = prediccionDF
datos_nuevos['prediccion_mora'].value_counts()